In [1]:
import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 90% !important; }</style>'))

In [2]:
import pandas as pd
import seaborn  as sns
import numpy as np
import matplotlib.pyplot as plt
import pybedtools
import os

# Loading genetics

In [114]:
genetics = pd.read_csv("CAD_SNPs/rsids.imputed.tsv", sep='\t')

# Loading REgulamentary huvec results

In [136]:
# re = pd.read_csv("/project/Wellcome_Discovery/shared/REgulamentary/pipeline_output/huvec/merge/08_REgulamentary/mlv_REgulamentary.csv", sep='\t')
re = pd.read_csv("/project/Wellcome_Discovery/shared/REgulamentary/pipeline_output/astrocyte/merge/08_REgulamentary/mlv_REgulamentary.csv", sep='\t')

# Intersecting genetics with REgulamentary output

https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html

In [137]:
if not os.path.exists('tmp'):
    os.makedirs('tmp')
pybedtools.set_tempdir('tmp')

In [138]:
genetics_filt = genetics[['CHR_ID', 'CHR_POS', 'SNPS']]
genetics_filt.columns = ['chr', 'start', 'SNPS']
genetics_filt['end'] = genetics_filt['start']+1
genetics_filt = genetics_filt[['chr', 'start', 'end', 'SNPS']]

In [139]:
re_filt = re[['chromosome', 'start', 'end']]
re_filt.columns = ['chr', 'start', 'end']
re_filt = re_filt[['chr', 'start', 'end']]

In [140]:
genetics_pbt = pybedtools.BedTool.from_dataframe(genetics_filt)
re_pbt = pybedtools.BedTool.from_dataframe(re_filt)

In [141]:
intersect1 = genetics_pbt.intersect(re_pbt)
intersect1 = intersect1.to_dataframe().sort_values(['chrom', 'start'])
intersect1.reset_index(inplace=True, drop=True)

intersect2 = re_pbt.intersect(genetics_pbt, wa=True)
intersect2 = intersect2.to_dataframe().sort_values(['chrom', 'start'])
intersect2.reset_index(inplace=True, drop=True)

In [142]:
intersect_final = intersect2
intersect_final['SNPS'] = intersect1['name']
intersect_final['SNP_chrom'] = intersect1['chrom']
intersect_final['SNP_start'] = intersect1['start']
intersect_final.index = intersect_final['chrom'].astype(str)+":"+intersect_final['start'].astype(str)+":"+intersect_final['end'].astype(str)

In [143]:
re_filt2 = re
re_filt2.index = re['chromosome'].astype(str)+":"+re['start'].astype(str)+":"+re['end'].astype(str)
re_filt2 = re_filt2[re_filt2.index.isin(intersect_final.index.tolist())]

In [144]:
dict_re = {}
for i, j in zip(re_filt2.index, re_filt2['RE']):
    dict_re[i] = j

re_list = []
for i in intersect_final.index:
    re_list.append(dict_re[i])
    
intersect_final['RE'] = re_list

In [145]:
intersect_final

,chrom,start,end,SNPS,SNP_chrom,SNP_start,RE
chr1:26695244:26696166,chr1,26695244,26696166,rs114165349,chr1,26695422,Promoter
chr1:26852919:26853633,chr1,26852919,26853633,rs75460349,chr1,26853597,CTCF
chr1:37989711:37990624,chr1,37989711,37990624,rs4360494,chr1,37990219,Promoter/CTCF
chr1:37989711:37990624,chr1,37989711,37990624,rs4360494,chr1,37990219,Promoter/CTCF
chr1:37989817:37990510,chr1,37989817,37990510,rs4072980,chr1,37990434,Promoter/CTCF
...,...,...,...,...,...,...,...
chr9:129145669:129146995,chr9,129145669,129146995,rs1360172,chr9,129145701,Enhancer
chr9:129145669:129146995,chr9,129145669,129146995,rs1819381,chr9,129145728,Enhancer
chr9:129174602:129176485,chr9,129174602,129176485,rs141780496,chr9,129175773,Promoter/CTCF
chr9:129177640:129178615,chr9,129177640,129178615,rs184457,chr9,129177740,Promoter


In [146]:
df = pd.DataFrame(intersect_final['RE'].value_counts())
df['RE'] = df.index
df.index.name = None

In [147]:
df

,count,RE
Promoter,178,Promoter
CTCF,140,CTCF
Enhancer,70,Enhancer
Promoter/CTCF,29,Promoter/CTCF
Enhancer/CTCF,10,Enhancer/CTCF


In [148]:
import plotly.express as px
fig = px.pie(df, values='count', names='RE', title='RE')
fig.show()